In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image

In [2]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

print(ALL_CHAR_SET.index('NONE'))

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot


36


In [3]:
'''Define TRAIN_SIZE and TEST_SIZE'''
TRAIN_SIZE = 10000
TEST_SIZE = 1000

In [4]:
# modified dataset class
class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:TRAIN_SIZE]
            self.img.sort(key = lambda x :(int(x[:-4]))) # 데이터 정렬
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:TRAIN_SIZE]
            print(self.img, self.labels)
        else: 
            self.img = os.listdir(self.path)[:TEST_SIZE]
            self.img.sort(key = lambda x :(int(x[:-4]))) # 데이터 정렬
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:TEST_SIZE]
        
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        img = img.convert('L')
        label = self.labels[idx]
        label_oh = []
        # one-hot for each character
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor(),
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

    # normalize: unify the feature sizes 
    transforms.Normalize((0.0070, ), (0.0011, )) # 평균, 표준편차

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
])

In [5]:
from google.colab import drive
drive.mount('/content/drive') # 데이터 로드를 위해 드라이브 접근

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
gPath = '/content/drive/My Drive/ML_final/'

from sklearn.model_selection import train_test_split

train_ds = Mydataset(gPath+'Data/train', gPath+'Data/train.txt',transform=transform)
# overfitting 방지를 위해 train set, validation set 을 나눔. 비율 = 9:1
train_ds, val_ds = train_test_split(train_ds, test_size=0.1, random_state=42, shuffle=True)
test_ds = Mydataset(gPath+'Data/test', gPath+'Data/test.txt',False, transform)

train_dl = DataLoader(train_ds, batch_size=128, num_workers=4)
val_dl = DataLoader(val_ds, batch_size=128, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)

print("train_ds shape: {}".format(len(train_ds)))
print("val_ds shape: {}".format(len(val_ds)))
print("test_ds shape: {}".format(len(test_ds)))


['0.png', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png', '56.png', '57.png', '58.png', '59.png', '60.png', '61.png', '62.png', '63.png', '64.png', '65.png', '66.png', '67.png', '68.png', '69.png', '70.png', '71.png', '72.png', '73.png', '74.png', '75.png', '76.png', '77.png', '78.png', '79.png', '80.png', '81.png', '82.png', '83.png', '84.png', '85.png', '86.png', '87.png', '88.png', '89.png', '90.png', '91.png', '92.png', '93.png', '94.png', '95.png', '96.png', '97.png', '98.png', '99.png', '100.png'

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# ''' 데이터 transform 시 normalize를 위해 평균, 표준편차 계산하는 코드
#     계산된 결과를 위의 transform 안에 넣음 '''
# mean = 0. # 초기화
# std = 0.
# for images, _, _ in train_dl:
#     batch_samples = images.size(0) # batch size 
#     images = images.view(batch_samples, images.size(1), -1) # shape 처리
#     mean += images.mean(2).sum(0) # 평균 계산
#     std += images.std(2).sum(0) # 표준편차 계산

# mean /= len(train_dl.dataset)
# std /= len(train_dl.dataset)
# print(mean, std)

In [7]:
"""To CUDA for local run"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:0


In [9]:
#CNN
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
'''Problem2.2 Write your code to import pre-trained ResNet'''

torch.cuda.empty_cache()

from torchsummary import summary # 모델의 구조를 확인하기 위한 라이브러리
import resnet # resnet.py import 

Net = resnet.resnet18(pretrained=True) # pretrain = True 
# in_channels=1
Net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 
""" pretrained된 ResNet의 fc layer은 1000 out_features를 디폴트로 가짐
    프로젝트에 맞도록 ALL_CHAR_SET_LEN*MAX_CAPTCHA로 수정 """ 
Net.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)
# Net.fc = nn.Sequential( # dropout 추가한 fc layer
#     nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True),
#     nn.Dropout(0.25),
# )
Net = Net.to(device)

summary(Net, input_size=(1, 160, 60))
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 80, 30]           3,136
       BatchNorm2d-2           [-1, 64, 80, 30]             128
              ReLU-3           [-1, 64, 80, 30]               0
         MaxPool2d-4           [-1, 64, 40, 15]               0
            Conv2d-5           [-1, 64, 40, 15]          36,864
       BatchNorm2d-6           [-1, 64, 40, 15]             128
              ReLU-7           [-1, 64, 40, 15]               0
            Conv2d-8           [-1, 64, 40, 15]          36,864
       BatchNorm2d-9           [-1, 64, 40, 15]             128
             ReLU-10           [-1, 64, 40, 15]               0
       BasicBlock-11           [-1, 64, 40, 15]               0
           Conv2d-12           [-1, 64, 40, 15]          36,864
      BatchNorm2d-13           [-1, 64, 40, 15]             128
             ReLU-14           [-1, 64,

In [ ]:
#LSTM 
class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, vocab_size, num_layers=1):
        super(LSTM, self).__init__()
    
        self.cnn_dim = cnn_dim
        self.hidden_size = hidden_size   
        self.vocab_size = vocab_size
        
        # Building LSTM cell
        self.lstm_cell = nn.LSTMCell(input_size=self.vocab_size, hidden_size=hidden_size)
    
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        '''Problem2.1 : Connect CNN model to LSTM model'''
        
        # output fully connected layer
        self.fc_in = nn.Linear(in_features=self.cnn_dim, out_features=self.vocab_size) # 512, 37
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size) # 128, 37
        
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################
    
        # embedding layer
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.vocab_size)
    
        # activations
        self.softmax = nn.Softmax(dim=1)

    def forward(self, features, captions):

        batch_size = features.size(0)
        cnn_dim = features.size(1)

        hidden_state = torch.zeros((batch_size, self.hidden_size)).cuda()
        cell_state = torch.zeros((batch_size, self.hidden_size)).cuda()

        # define the output tensor placeholder
        outputs = torch.zeros((batch_size, captions.size(1), self.vocab_size)).cuda()  # 128 * 7 * 37

        # embed the captions
        captions_embed = self.embed(captions)  

        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        '''Problem1: Design LSTM model for captcha image recognition'''

        avgpool = nn.AdaptiveAvgPool2d(1)  # feature의 첫 shape: 128 x 512 x 5 x 2
        features = avgpool(features)  # avgpool을 거친 후: 128 x 512 x 1 x 1
        features = torch.flatten(features, 1)  # torch flatten 결과: 128 x 512
        features = self.fc_in(features)  # fc_in layer로 vocab size 맞춰줌: 512 -> 37

        # for each word: 128 * 7 * 37
        for t in range(captions.size(1)):  # 0 ~ 6
            if t == 0:  # t = 0일 때 input = features
                hidden_state, cell_state = self.lstm_cell(features, (hidden_state, cell_state))
            else:  # t > 0일 때 input = label one hot vector
                hidden_state, cell_state = self.lstm_cell(captions[:, t, :], (hidden_state, cell_state))
            out = self.fc_out(hidden_state)  # 8 -> 37 predict next state
            outputs[:, t, :] = out
        outputs = self.softmax(outputs) # 최종 softmax activation

        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################
        return outputs

In [10]:
cnn_dim=512   #resnet18-512
hidden_size=128
vocab_size=37     #ALL_CHAR_SET_LEN

lstm = LSTM(cnn_dim=cnn_dim, hidden_size=hidden_size, vocab_size=vocab_size)
lstm = lstm.to(device)

params = list(Net.parameters()) + list(lstm.parameters()) # cnn과 lstm 연결을 위해 parameter 합함

'''DEFINE loss, optimizer'''
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
loss_func = nn.MultiLabelSoftMarginLoss()
cnn_optim = torch.optim.Adam(Net.parameters(), lr= 0.001) # ResNet 만 사용 시 optimizer
lstm_optim = torch.optim.Adam(params, lr=0.001) # use_lstm = True
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [11]:
# overfitting 방지를 위해 EarlyStopping 도입
class EarlyStopping: 
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'): 
        """
        Args:
            patience: validation loss가 개선된 후 기다리는 기간, default는 7
            path: checkpoint 저장 경로 
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss # 이전 epoch의 val_loss와 비교해 score 측정

        if self.best_score is None:  # 가장 좋은 score 저장 
            self.best_score = score
            self.save_checkpoint(val_loss, model)  # checkpoint에 기록
        elif score < self.best_score + self.delta:  # valid loss 가 높아지면 counter 1 증가
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True  # early stop 발생
        else:  # 가장 좋은 score 저장 후 counter 0으로 초기화
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """validation loss가 감소하면 모델을 저장한다."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
"""TRAINING"""
print_interval = 15
max_epoch = 50
use_lstm = True

train_losses = [] # training loss track
valid_losses = [] # validation loss track
avg_train_losses = [] # average training loss per epoch 
avg_valid_losses = [] # average validation loss per epoch
accuracy_list = [] # accuracy per epoch

# initialize the early_stopping object
early_stopping = EarlyStopping(patience=max_epoch/5, verbose=True)

for epoch in range(max_epoch):
    total = 0
    train_acc = 0.0

    ###################
    # train the model #
    ###################
    Net.train() # 모델 train 단계

    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda() 
        batch_size, _ = label_oh.shape
        pred, feature = Net(img) # ResNet

        if use_lstm:
          # lstm 에 feature과 captions를 input시키기 위한 reshape
          label_oh = label_oh.reshape(batch_size, MAX_CAPTCHA, ALL_CHAR_SET_LEN)  # (128, 259) -> (128, 7, 37)
          pred = lstm(feature, label_oh) # Net(CNN)의 결과인 feature를 lstm의 input으로 넣어줌

          loss = loss_func(pred, label_oh) # loss 계산

          lstm_optim.zero_grad() # gradients clear
          loss.backward() # compute gradient of the loss with respect to model parameters
          lstm_optim.step() # parameter를 업데이트, optimization 한 스텝 수행

        else:
          loss = loss_func(pred, label_oh) # calculate the loss

          cnn_optim.zero_grad() # gradients clear
          loss.backward() # compute gradient of the loss with respect to model parameters
          cnn_optim.step() # parameter를 업데이트, optimization 한 스텝 수행

        train_losses.append(loss.item())  # training loss 기록

        '''train accuracy 측정'''
        pred = pred.squeeze(0)
        label_oh = label_oh.squeeze(0)
        for i in range(3): # pred와 label_oh를 비교할 수 있는 형태로 처리
            if pred.cpu().tolist()[i][0:1] : c0 = np.argmax(pred.cpu().tolist()[i][0:1])
            if pred.cpu().tolist()[i][1:2] : c1 = np.argmax(pred.cpu().tolist()[i][1:2])
            if pred.cpu().tolist()[i][2:3] : c2 = np.argmax(pred.cpu().tolist()[i][2:3])
            if pred.cpu().tolist()[i][3:4] : c3 = np.argmax(pred.cpu().tolist()[i][3:4])
            if pred.cpu().tolist()[i][4:5] : c4 = np.argmax(pred.cpu().tolist()[i][4:5])
            if pred.cpu().tolist()[i][5:6] : c5 = np.argmax(pred.cpu().tolist()[i][5:6])
            if pred.cpu().tolist()[i][6:7] : c6 = np.argmax(pred.cpu().tolist()[i][6:7])
        
            if label_oh.cpu().tolist()[i][0:1] :d0 = np.argmax(label_oh.cpu().tolist()[i][0:1])
            if label_oh.cpu().tolist()[i][1:2] :d1 = np.argmax(label_oh.cpu().tolist()[i][1:2])
            if label_oh.cpu().tolist()[i][2:3] :d2 = np.argmax(label_oh.cpu().tolist()[i][2:3])
            if label_oh.cpu().tolist()[i][3:4] :d3 = np.argmax(label_oh.cpu().tolist()[i][3:4])
            if label_oh.cpu().tolist()[i][4:5] :d4 = np.argmax(label_oh.cpu().tolist()[i][4:5])
            if label_oh.cpu().tolist()[i][5:6] :d5 = np.argmax(label_oh.cpu().tolist()[i][5:6])
            if label_oh.cpu().tolist()[i][6:7] :d6 = np.argmax(label_oh.cpu().tolist()[i][6:7])

            # 각 char가 동일한 경우 acc에 카운트 반영
            train_acc += int(c0==d0)+int(c1==d1)+int(c2==d2)+int(c3==d3)+int(c4==d4)+int(c5==d5)+int(c6==d6)
            total += 7

    ######################    
    # validate the model #
    ######################
    Net.eval() # validation(evaluation) 단계
    for step, i in enumerate(val_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda() 
        batch_size, _ = label_oh.shape
        pred, feature = Net(img) 
        
        if use_lstm:
          label_oh = label_oh.reshape(batch_size, MAX_CAPTCHA, ALL_CHAR_SET_LEN)  # (128, 259) -> (128, 7, 37)
          pred = lstm(feature, label_oh)

        loss = loss_func(pred, label_oh) # loss 계산
        valid_losses.append(loss.item()) # validation loss 기록


    train_loss = np.average(train_losses) # epoch마다의 train loss 평균을 구함
    valid_loss = np.average(valid_losses) # epoch마다의 validation loss 평균을 구함
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)

    accuracy = 100 * train_acc / total # 해당 epoch의 train accuracy 최종 계산, 기록
    accuracy_list.append(accuracy)
    
    # epoch의 train_loss, valid_loss, accuracy 출력
    epoch_len = len(str(epoch))
    print_msg = (f'[{epoch:>{epoch_len}}/{epoch:>{epoch_len}}] ' +
                  f'train_loss: {train_loss:.5f} ' +
                  f'valid_loss: {valid_loss:.5f} ' + 
                  f'accuracy: {accuracy:.5f}')
    print(print_msg)
    
    # clear lists to track next epoch
    train_losses = []
    valid_losses = []
    
    # early stopping 으로 validation loss가 감소했는지 체크
    early_stopping(valid_loss, Net)
    
    if early_stopping.early_stop: # 얼리스탑 발생 시 train 중지
        print("Early stopping")
        break

In [ ]:
''' 모델 train 한 결과 loss와 accuracy를 시각화 '''
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(avg_train_losses)+1),avg_train_losses, label='Training Loss')
plt.plot(range(1,len(avg_valid_losses)+1),avg_valid_losses,label='Validation Loss')

minposs = avg_valid_losses.index(min(avg_valid_losses))+1  # 가장 낮은 validation loss 위치를 찾음
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint') # 해당 위치가 early stop된 체크포인트

plt.xlabel('epochs') # loss 그래프
plt.ylabel('loss')
plt.xlim(0, len(avg_train_losses)+1) 
plt.grid(True)
plt.legend()
plt.show()


plt.plot(accuracy_list) # accuracy 그래프
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
use_lstm = True
tf_rate = 0
char_correct = 0
word_correct = 0
total = 0

Net.eval()
lstm.eval()

def get_str(ch_arr):
    ch_str = ''
    for ch in ch_arr:
        if ch == 'NONE':
            ch_str = ch_str + '_'
        else:
            ch_str = ch_str + ch
    return ch_str

with torch.no_grad(): 
    for step, (img, label_oh, label) in enumerate(test_dl):
        img = Variable(img).cuda()  
        label_oh = Variable(label_oh.float()).cuda()
        pred, feature = Net(img)
        
        if use_lstm:
            label_oh = label_oh.reshape(1, MAX_CAPTCHA, ALL_CHAR_SET_LEN)
            pred = lstm(feature, label_oh)

        pred = pred.squeeze(0)
        label_oh = label_oh.squeeze(0)
        c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:1])]
        c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1:1*2])]
        c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1*2:1*3])]
        c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1*3:1*4])]
        c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1*4:1*5])]
        c5 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1*5:1*6])]
        c6 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[1*6:1*7])]
    
        d0 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[0:1])]
        d1 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1:1*2])]
        d2 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1*2:1*3])]
        d3 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1*3:1*4])]
        d4 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1*4:1*5])]
        d5 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1*5:1*6])]
        d6 = ALL_CHAR_SET[np.argmax(label_oh.cpu().tolist()[1*6:1*7])]
        
        c_arr = (c0, c1, c2, c3, c4, c5, c6)
        d_arr = (d0, d1, d2, d3, d4, d5, d6)
    
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
    
        c = get_str(c_arr)
        d = get_str(d_arr)

        char_correct += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        word_correct += (c==d)
        total += 1
        
        print('label:', d, ', pred:', c)
        
print(100/7*char_correct/total)
print(100*word_correct/total)

"""END TEST"""